In [75]:
from IPython.display import Latex
from IPython.display import display, Math, Latex
print_latex = lambda latex_str: display(Latex(latex_str))

print_latex(r'$\left\{\begin{array}{l}R(s)=\left\{\begin{array}{l}+1,\ s=(4,3)\\-1,\ s=(4,2)\\-0.02\ ,\ otherwise.\end{array}\right.\\\
    \gamma=0.99\\\
    正常走概率0.8, 向左向右0.1\
    \end{array}\right.$')


<IPython.core.display.Latex object>

In [1]:
# 价值迭代算法

import pandas as pd
import numpy as np
import pprint

max_x = 3
max_y = 2
min_x = 0
min_y = 0
direc = ['u', 'r', 'd', 'l']
a_size = len(direc)
offset = {'u': (0, 1), 'd': (0, -1), 'l': (-1, 0), 'r': (1, 0)}
p_normal = 0.8
p_abnormal = 0.1
R_rest = -0.02
gamma = 0.99
e = 1e-30
obstacle = {(1, 1)}
states = {(x, y) for x in range(min_x, max_x+1) for y in range(min_y, max_y+1)} - obstacle
next_state = {}
for s in states:
    if s[0] != 3 or s[1] != 2:
        for a in direc:
            if a == 'u' and s[1] == max_y or a == 'd' and s[1] == min_y or a == 'l' and s[0] == min_x or a == 'r' and s[0] == max_x:
                next_state[s + (a,)] = s
            else: 
                if tuple(np.add(s, offset[a])) in obstacle:
                    next_state[s + (a,)] = s
                else:
                    next_state[s + (a,)] = tuple(np.add(s, offset[a]))
    else:
        for a in direc:
            next_state[s + (a,)] = s

            
# Reward Function
R = np.ones((max_x-min_x+1, max_y-min_y+1)) * R_rest
R[3][2] = 1
R[3][1] = -1
Rs = {s: R[s[0]][s[1]] for s in states}
Rs.update({s: 0 for s in obstacle})

# Value Function
V = np.zeros((max_x-min_x+1, max_y-min_y+1))
Vs = {s: V[s[0]][s[1]] for s in states}
Vs.update({s: -np.inf for s in obstacle})

# Get Value Function
n = 0
while True:
    lastV = pd.Series(Vs)
    for s in states:
        max_next = -np.inf
        if s == (3, 2):
            max_next = 0
        else:
            for i in range(len(direc)):
                cur_next = p_normal * Vs[next_state[s + (direc[i],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+1)%a_size],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+a_size-1)%a_size],)]]
                if cur_next > max_next:
                    max_next = cur_next
        Vs[s] = Rs[s] + gamma*max_next
    V = pd.Series(Vs)
    n += 1
    if np.sum(np.square(lastV-V)) < e:
        break
    
# Get Policy
policy = {}
for s in states:
        max_next = -np.inf
        best_policy = direc[0]
        if s == (3, 2):
            policy[s] = 'Stop'
        else:
            for i in range(len(direc)):
                cur_next = p_normal * Vs[next_state[s + (direc[i],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+1)%a_size],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+a_size-1)%a_size],)]]
                if cur_next > max_next:
                    best_policy = direc[i]
                    max_next = cur_next
            policy[s] = best_policy

print(n)
V = np.array([[Vs[(x, y)] for x in range(min_x, max_x+1)] for y in range(min_y, max_y+1)])[::-1]
print(V)
Po = np.array([[policy[(x, y)] if (x, y) not in obstacle else None for x in range(min_x, max_x+1)] for y in range(min_y, max_y+1)])[::-1]
print(Po)


38
[[ 0.8664417   0.90710179  0.94380762  1.        ]
 [ 0.83070053        -inf  0.7916229  -0.14387273]
 [ 0.79110902  0.75630717  0.74271026  0.61485363]]
[['r' 'r' 'r' 'Stop']
 ['u' None 'u' 'u']
 ['u' 'l' 'u' 'l']]


In [ ]:
# 价值迭代算法

import pandas as pd
import numpy as np
import pprint

max_x = 3
max_y = 2
min_x = 0
min_y = 0
direc = ['u', 'r', 'd', 'l']
a_size = len(direc)
offset = {'u': (0, 1), 'd': (0, -1), 'l': (-1, 0), 'r': (1, 0)}
p_normal = 0.8
p_abnormal = 0.1
R_rest = -0.02
gamma = 0.99
e = 1e-30
obstacle = {(1, 1)}
states = {(x, y) for x in range(min_x, max_x+1) for y in range(min_y, max_y+1)} - obstacle
next_state = {}
for s in states:
    if s[0] != 3 or s[1] != 2:
        for a in direc:
            if a == 'u' and s[1] == max_y or a == 'd' and s[1] == min_y or a == 'l' and s[0] == min_x or a == 'r' and s[0] == max_x:
                next_state[s + (a,)] = s
            else: 
                if tuple(np.add(s, offset[a])) in obstacle:
                    next_state[s + (a,)] = s
                else:
                    next_state[s + (a,)] = tuple(np.add(s, offset[a]))
    else:
        for a in direc:
            next_state[s + (a,)] = s

            
# Reward Function
R = np.ones((max_x-min_x+1, max_y-min_y+1)) * R_rest
R[3][2] = 1
R[3][1] = -1
Rs = {s: R[s[0]][s[1]] for s in states}
Rs.update({s: 0 for s in obstacle})

# Value Function
V = np.zeros((max_x-min_x+1, max_y-min_y+1))
Vs = {s: V[s[0]][s[1]] for s in states}
Vs.update({s: -np.inf for s in obstacle})

# Get Value Function
n = 0
while True:
    lastV = pd.Series(Vs)
    for s in states:
        max_next = -np.inf
        if s == (3, 2):
            max_next = 0
        else:
            for i in range(len(direc)):
                cur_next = p_normal * Vs[next_state[s + (direc[i],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+1)%a_size],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+a_size-1)%a_size],)]]
                if cur_next > max_next:
                    max_next = cur_next
        Vs[s] = Rs[s] + gamma*max_next
    V = pd.Series(Vs)
    n += 1
    if np.sum(np.square(lastV-V)) < e:
        break
    
# Get Policy
policy = {}
for s in states:
        max_next = -np.inf
        best_policy = direc[0]
        if s == (3, 2):
            policy[s] = 'Stop'
        else:
            for i in range(len(direc)):
                cur_next = p_normal * Vs[next_state[s + (direc[i],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+1)%a_size],)]] \
                            + p_abnormal*Vs[next_state[s + (direc[(i+a_size-1)%a_size],)]]
                if cur_next > max_next:
                    best_policy = direc[i]
                    max_next = cur_next
            policy[s] = best_policy

print(n)
V = np.array([[Vs[(x, y)] for x in range(min_x, max_x+1)] for y in range(min_y, max_y+1)])[::-1]
print(V)
Po = np.array([[policy[(x, y)] if (x, y) not in obstacle else None for x in range(min_x, max_x+1)] for y in range(min_y, max_y+1)])[::-1]
print(Po)
